# Project 18: Reinforcement Learning Game

**Create an AI that learns and masters simple games**

In this tutorial, we'll build AI agents that learn to play games through trial and error - the essence of Reinforcement Learning (RL).

**What is Reinforcement Learning?**

RL is a machine learning paradigm where an **agent** learns to make decisions by interacting with an **environment** to maximize cumulative **rewards**.

```
    ┌─────────────────────────────────────────┐
    │                                         │
    │    State (S_t)          Reward (R_t)    │
    │        │                    ▲           │
    │        ▼                    │           │
    │    ┌───────┐          ┌─────────────┐   │
    │    │ AGENT │──Action──│ ENVIRONMENT │   │
    │    └───────┘   (A_t)  └─────────────┘   │
    │                                         │
    └─────────────────────────────────────────┘
```

**Algorithms we'll implement:**
1. Q-Learning (tabular)
2. Deep Q-Network (DQN)
3. Policy Gradient (REINFORCE)
4. DQN for Atari-style games

## Table of Contents

1. [Setup and Installation](#1-setup-and-installation)
2. [RL Fundamentals](#2-rl-fundamentals)
3. [GridWorld Environment](#3-gridworld-environment)
4. [Q-Learning (Tabular)](#4-q-learning-tabular)
5. [Deep Q-Network (DQN)](#5-deep-q-network-dqn)
6. [CartPole with DQN](#6-cartpole-with-dqn)
7. [Policy Gradient (REINFORCE)](#7-policy-gradient-reinforce)
8. [Pong-Style Game with DQN](#8-pong-style-game-with-dqn)
9. [Training Visualization](#9-training-visualization)
10. [Summary](#10-summary)

## 1. Setup and Installation

In [ ]:
# Install required packages
!pip install -q gymnasium torch numpy matplotlib
!pip install -q gymnasium[classic_control]  # For CartPole, etc.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import deque, namedtuple
import random
from typing import List, Tuple, Optional
from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')

# Deep Learning
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# RL Environment
import gymnasium as gym

# Set seeds for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"PyTorch version: {torch.__version__}")
print(f"Gymnasium version: {gym.__version__}")
print(f"Device: {device}")

## 2. RL Fundamentals

### Key Concepts

| Concept | Description | Example (Pong) |
|---------|-------------|----------------|
| **State (S)** | Current situation | Ball position, paddle position |
| **Action (A)** | What agent can do | Move up, down, stay |
| **Reward (R)** | Feedback signal | +1 for scoring, -1 for losing |
| **Policy (π)** | Strategy for choosing actions | Neural network |
| **Value (V)** | Expected future rewards | How good is this state? |
| **Q-Value (Q)** | Value of action in state | How good is this action here? |

### The RL Loop

```python
state = env.reset()
while not done:
    action = agent.select_action(state)  # Policy
    next_state, reward, done = env.step(action)
    agent.learn(state, action, reward, next_state)  # Update
    state = next_state
```

### Exploration vs Exploitation

- **Exploration**: Try new actions to discover better strategies
- **Exploitation**: Use known good actions to maximize reward
- **ε-greedy**: With probability ε explore, otherwise exploit

In [ ]:
# Visualize exploration vs exploitation tradeoff
def epsilon_decay_schedule(episodes, epsilon_start=1.0, epsilon_end=0.01, decay_rate=0.995):
    """Calculate epsilon values over episodes."""
    epsilons = []
    epsilon = epsilon_start
    for _ in range(episodes):
        epsilons.append(epsilon)
        epsilon = max(epsilon_end, epsilon * decay_rate)
    return epsilons

# Plot different decay schedules
fig, ax = plt.subplots(figsize=(10, 5))

episodes = 500
for decay in [0.99, 0.995, 0.999]:
    epsilons = epsilon_decay_schedule(episodes, decay_rate=decay)
    ax.plot(epsilons, label=f'Decay = {decay}')

ax.axhline(y=0.1, color='r', linestyle='--', label='Common minimum')
ax.set_xlabel('Episode')
ax.set_ylabel('Epsilon (Exploration Rate)')
ax.set_title('Epsilon Decay Schedules: Exploration → Exploitation')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

print("Early episodes: High ε = More exploration (random actions)")
print("Later episodes: Low ε = More exploitation (learned actions)")

## 3. GridWorld Environment

Let's start with a simple custom environment: GridWorld.

```
┌───┬───┬───┬───┐
│ S │   │   │ G │  S = Start, G = Goal (+1)
├───┼───┼───┼───┤
│   │ X │   │   │  X = Obstacle (-1)
├───┼───┼───┼───┤
│   │   │   │   │
└───┴───┴───┴───┘
```

- **State**: (row, col) position
- **Actions**: Up, Down, Left, Right
- **Rewards**: +1 at goal, -1 at obstacle, -0.01 per step

In [ ]:
class GridWorld:
    """
    Simple GridWorld environment for learning RL basics.
    
    The agent starts at position (0,0) and must reach the goal
    while avoiding obstacles.
    """
    
    # Actions
    UP = 0
    DOWN = 1
    LEFT = 2
    RIGHT = 3
    
    ACTION_NAMES = ['UP', 'DOWN', 'LEFT', 'RIGHT']
    
    def __init__(self, rows=4, cols=4):
        self.rows = rows
        self.cols = cols
        self.n_states = rows * cols
        self.n_actions = 4
        
        # Define special cells
        self.start = (0, 0)
        self.goal = (0, cols - 1)
        self.obstacles = [(1, 1)]  # Obstacles
        
        # Current position
        self.agent_pos = self.start
        
    def reset(self):
        """Reset environment to initial state."""
        self.agent_pos = self.start
        return self._get_state()
    
    def _get_state(self):
        """Convert (row, col) to state index."""
        return self.agent_pos[0] * self.cols + self.agent_pos[1]
    
    def _pos_from_state(self, state):
        """Convert state index to (row, col)."""
        return (state // self.cols, state % self.cols)
    
    def step(self, action):
        """
        Take action and return (next_state, reward, done, info).
        """
        row, col = self.agent_pos
        
        # Calculate new position
        if action == self.UP:
            row = max(0, row - 1)
        elif action == self.DOWN:
            row = min(self.rows - 1, row + 1)
        elif action == self.LEFT:
            col = max(0, col - 1)
        elif action == self.RIGHT:
            col = min(self.cols - 1, col + 1)
        
        self.agent_pos = (row, col)
        
        # Calculate reward
        if self.agent_pos == self.goal:
            reward = 1.0
            done = True
        elif self.agent_pos in self.obstacles:
            reward = -1.0
            done = True
        else:
            reward = -0.01  # Small penalty for each step
            done = False
        
        return self._get_state(), reward, done, {}
    
    def render(self):
        """Visualize the grid."""
        grid = [['.' for _ in range(self.cols)] for _ in range(self.rows)]
        
        # Mark special cells
        grid[self.goal[0]][self.goal[1]] = 'G'
        for obs in self.obstacles:
            grid[obs[0]][obs[1]] = 'X'
        grid[self.agent_pos[0]][self.agent_pos[1]] = 'A'
        
        # Print grid
        print('┌' + '───┬' * (self.cols - 1) + '───┐')
        for i, row in enumerate(grid):
            print('│ ' + ' │ '.join(row) + ' │')
            if i < self.rows - 1:
                print('├' + '───┼' * (self.cols - 1) + '───┤')
        print('└' + '───┴' * (self.cols - 1) + '───┘')
        print(f"Agent: {self.agent_pos}, Goal: {self.goal}")

# Test the environment
env = GridWorld()
state = env.reset()

print("GridWorld Environment")
print("=" * 30)
print("A = Agent, G = Goal, X = Obstacle")
print()
env.render()

# Take some random steps
print("\nRandom exploration:")
for i in range(3):
    action = random.randint(0, 3)
    next_state, reward, done, _ = env.step(action)
    print(f"Action: {env.ACTION_NAMES[action]:5} → State: {next_state}, Reward: {reward:.2f}")
    if done:
        break

## 4. Q-Learning (Tabular)

Q-Learning is a model-free RL algorithm that learns the value of actions directly.

### Q-Value Update Rule (Bellman Equation)

$$Q(s, a) \leftarrow Q(s, a) + \alpha \left[ r + \gamma \max_{a'} Q(s', a') - Q(s, a) \right]$$

Where:
- $\alpha$ = learning rate
- $\gamma$ = discount factor (importance of future rewards)
- $r$ = immediate reward
- $s'$ = next state

### Algorithm

```
Initialize Q-table with zeros
For each episode:
    state = env.reset()
    While not done:
        action = ε-greedy(Q, state)
        next_state, reward, done = env.step(action)
        Q[state, action] += α * (reward + γ * max(Q[next_state]) - Q[state, action])
        state = next_state
```

In [ ]:
class QLearningAgent:
    """
    Tabular Q-Learning agent.
    
    Learns Q-values for each (state, action) pair using the
    Bellman equation update rule.
    """
    
    def __init__(self, n_states, n_actions, 
                 learning_rate=0.1, 
                 discount_factor=0.99,
                 epsilon_start=1.0,
                 epsilon_end=0.01,
                 epsilon_decay=0.995):
        """
        Initialize Q-Learning agent.
        
        Args:
            n_states: Number of states in environment
            n_actions: Number of possible actions
            learning_rate: How much to update Q-values (α)
            discount_factor: Importance of future rewards (γ)
            epsilon_*: Exploration parameters
        """
        self.n_states = n_states
        self.n_actions = n_actions
        self.lr = learning_rate
        self.gamma = discount_factor
        
        # Exploration parameters
        self.epsilon = epsilon_start
        self.epsilon_end = epsilon_end
        self.epsilon_decay = epsilon_decay
        
        # Initialize Q-table with zeros
        self.q_table = np.zeros((n_states, n_actions))
        
    def select_action(self, state):
        """
        Select action using ε-greedy policy.
        
        With probability ε: random action (explore)
        Otherwise: best action from Q-table (exploit)
        """
        if random.random() < self.epsilon:
            return random.randint(0, self.n_actions - 1)
        else:
            return np.argmax(self.q_table[state])
    
    def learn(self, state, action, reward, next_state, done):
        """
        Update Q-value using Bellman equation.
        
        Q(s,a) ← Q(s,a) + α[r + γ·max(Q(s')) - Q(s,a)]
        """
        # Current Q-value
        current_q = self.q_table[state, action]
        
        # Target Q-value
        if done:
            target_q = reward  # No future rewards if done
        else:
            target_q = reward + self.gamma * np.max(self.q_table[next_state])
        
        # Update Q-value
        self.q_table[state, action] += self.lr * (target_q - current_q)
    
    def decay_epsilon(self):
        """Decay exploration rate."""
        self.epsilon = max(self.epsilon_end, self.epsilon * self.epsilon_decay)
    
    def get_policy(self):
        """Get optimal action for each state."""
        return np.argmax(self.q_table, axis=1)


def train_q_learning(env, agent, n_episodes=500, max_steps=100):
    """
    Train Q-Learning agent on environment.
    
    Returns:
        list: Episode rewards
    """
    episode_rewards = []
    
    for episode in range(n_episodes):
        state = env.reset()
        total_reward = 0
        
        for step in range(max_steps):
            # Select and perform action
            action = agent.select_action(state)
            next_state, reward, done, _ = env.step(action)
            
            # Learn from experience
            agent.learn(state, action, reward, next_state, done)
            
            total_reward += reward
            state = next_state
            
            if done:
                break
        
        # Decay exploration
        agent.decay_epsilon()
        episode_rewards.append(total_reward)
        
        # Print progress
        if (episode + 1) % 100 == 0:
            avg_reward = np.mean(episode_rewards[-100:])
            print(f"Episode {episode + 1}/{n_episodes} | "
                  f"Avg Reward: {avg_reward:.3f} | "
                  f"Epsilon: {agent.epsilon:.3f}")
    
    return episode_rewards

# Train Q-Learning agent on GridWorld
print("Training Q-Learning Agent on GridWorld")
print("=" * 50)

env = GridWorld()
agent = QLearningAgent(
    n_states=env.n_states,
    n_actions=env.n_actions,
    learning_rate=0.1,
    discount_factor=0.99
)

rewards = train_q_learning(env, agent, n_episodes=500)

In [ ]:
# Visualize training progress
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot rewards
ax1 = axes[0]
ax1.plot(rewards, alpha=0.3, label='Episode Reward')
# Moving average
window = 50
moving_avg = np.convolve(rewards, np.ones(window)/window, mode='valid')
ax1.plot(range(window-1, len(rewards)), moving_avg, label=f'{window}-Episode Moving Avg')
ax1.set_xlabel('Episode')
ax1.set_ylabel('Total Reward')
ax1.set_title('Q-Learning Training Progress')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Visualize Q-table as heatmap
ax2 = axes[1]
q_max = np.max(agent.q_table, axis=1).reshape(env.rows, env.cols)
im = ax2.imshow(q_max, cmap='YlOrRd')
ax2.set_title('Learned Value Function (Max Q per State)')
ax2.set_xlabel('Column')
ax2.set_ylabel('Row')
plt.colorbar(im, ax=ax2, label='Max Q-Value')

# Annotate with values
for i in range(env.rows):
    for j in range(env.cols):
        ax2.text(j, i, f'{q_max[i, j]:.2f}', ha='center', va='center', fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
# Visualize learned policy
print("Learned Policy (Best Action per State):")
print("=" * 40)

policy = agent.get_policy()
action_arrows = ['↑', '↓', '←', '→']

print('┌' + '───┬' * (env.cols - 1) + '───┐')
for i in range(env.rows):
    row_str = '│'
    for j in range(env.cols):
        state = i * env.cols + j
        if (i, j) == env.goal:
            cell = ' G '
        elif (i, j) in env.obstacles:
            cell = ' X '
        else:
            cell = f' {action_arrows[policy[state]]} '
        row_str += cell + '│'
    print(row_str)
    if i < env.rows - 1:
        print('├' + '───┼' * (env.cols - 1) + '───┤')
print('└' + '───┴' * (env.cols - 1) + '───┘')

print("\n↑=Up, ↓=Down, ←=Left, →=Right")
print("G=Goal, X=Obstacle")

In [ ]:
# Test the learned policy
print("\nTesting Learned Policy:")
print("=" * 40)

env = GridWorld()
state = env.reset()
total_reward = 0
path = [env.agent_pos]

print("Starting position:")
env.render()

for step in range(20):
    # Use learned policy (no exploration)
    action = np.argmax(agent.q_table[state])
    next_state, reward, done, _ = env.step(action)
    total_reward += reward
    path.append(env.agent_pos)
    
    print(f"\nStep {step + 1}: {env.ACTION_NAMES[action]}")
    env.render()
    
    if done:
        print(f"\n{'='*40}")
        print(f"Episode finished! Total reward: {total_reward:.2f}")
        print(f"Path: {' → '.join([str(p) for p in path])}")
        break
    
    state = next_state

## 5. Deep Q-Network (DQN)

For environments with large or continuous state spaces, we can't use a table. Instead, we use a **neural network** to approximate Q-values.

### Key DQN Innovations

1. **Experience Replay**: Store transitions and sample randomly to break correlation
2. **Target Network**: Separate network for stable Q-targets, updated periodically
3. **Neural Network**: Approximate Q(s, a) instead of table lookup

### Architecture

```
State → [Neural Network] → Q-values for all actions
         (FC layers)       [Q(s,a1), Q(s,a2), ...]
```

In [ ]:
# Experience replay buffer
Transition = namedtuple('Transition', ('state', 'action', 'reward', 'next_state', 'done'))

class ReplayBuffer:
    """
    Experience Replay Buffer.
    
    Stores transitions and samples random batches for training.
    This breaks correlation between consecutive samples.
    """
    
    def __init__(self, capacity=10000):
        self.buffer = deque(maxlen=capacity)
    
    def push(self, state, action, reward, next_state, done):
        """Add transition to buffer."""
        self.buffer.append(Transition(state, action, reward, next_state, done))
    
    def sample(self, batch_size):
        """Sample random batch of transitions."""
        transitions = random.sample(self.buffer, batch_size)
        return Transition(*zip(*transitions))
    
    def __len__(self):
        return len(self.buffer)


class DQN(nn.Module):
    """
    Deep Q-Network.
    
    Takes state as input and outputs Q-values for all actions.
    """
    
    def __init__(self, state_dim, action_dim, hidden_dim=128):
        super(DQN, self).__init__()
        
        self.network = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, action_dim)
        )
    
    def forward(self, x):
        return self.network(x)


class DQNAgent:
    """
    DQN Agent with experience replay and target network.
    """
    
    def __init__(self, state_dim, action_dim,
                 learning_rate=1e-3,
                 discount_factor=0.99,
                 epsilon_start=1.0,
                 epsilon_end=0.01,
                 epsilon_decay=0.995,
                 buffer_size=10000,
                 batch_size=64,
                 target_update_freq=10):
        
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.gamma = discount_factor
        self.batch_size = batch_size
        self.target_update_freq = target_update_freq
        
        # Exploration
        self.epsilon = epsilon_start
        self.epsilon_end = epsilon_end
        self.epsilon_decay = epsilon_decay
        
        # Networks
        self.policy_net = DQN(state_dim, action_dim).to(device)
        self.target_net = DQN(state_dim, action_dim).to(device)
        self.target_net.load_state_dict(self.policy_net.state_dict())
        self.target_net.eval()  # Target network is not trained directly
        
        # Optimizer
        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=learning_rate)
        
        # Replay buffer
        self.memory = ReplayBuffer(buffer_size)
        
        # Training counter
        self.train_step = 0
    
    def select_action(self, state):
        """Select action using ε-greedy policy."""
        if random.random() < self.epsilon:
            return random.randint(0, self.action_dim - 1)
        
        with torch.no_grad():
            state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)
            q_values = self.policy_net(state_tensor)
            return q_values.argmax().item()
    
    def store_transition(self, state, action, reward, next_state, done):
        """Store transition in replay buffer."""
        self.memory.push(state, action, reward, next_state, done)
    
    def learn(self):
        """Update network using batch from replay buffer."""
        if len(self.memory) < self.batch_size:
            return None
        
        # Sample batch
        batch = self.memory.sample(self.batch_size)
        
        # Convert to tensors
        states = torch.FloatTensor(np.array(batch.state)).to(device)
        actions = torch.LongTensor(batch.action).unsqueeze(1).to(device)
        rewards = torch.FloatTensor(batch.reward).unsqueeze(1).to(device)
        next_states = torch.FloatTensor(np.array(batch.next_state)).to(device)
        dones = torch.FloatTensor(batch.done).unsqueeze(1).to(device)
        
        # Current Q-values
        current_q = self.policy_net(states).gather(1, actions)
        
        # Target Q-values (from target network)
        with torch.no_grad():
            next_q = self.target_net(next_states).max(1)[0].unsqueeze(1)
            target_q = rewards + (1 - dones) * self.gamma * next_q
        
        # Compute loss
        loss = F.mse_loss(current_q, target_q)
        
        # Optimize
        self.optimizer.zero_grad()
        loss.backward()
        # Gradient clipping for stability
        torch.nn.utils.clip_grad_norm_(self.policy_net.parameters(), 1.0)
        self.optimizer.step()
        
        # Update target network periodically
        self.train_step += 1
        if self.train_step % self.target_update_freq == 0:
            self.target_net.load_state_dict(self.policy_net.state_dict())
        
        return loss.item()
    
    def decay_epsilon(self):
        """Decay exploration rate."""
        self.epsilon = max(self.epsilon_end, self.epsilon * self.epsilon_decay)

print("DQN Agent implemented!")
print(f"Components:")
print(f"  - Policy Network: Learns Q-values")
print(f"  - Target Network: Stable Q-targets")
print(f"  - Replay Buffer: Breaks correlation")
print(f"  - ε-greedy: Exploration strategy")

## 6. CartPole with DQN

CartPole is a classic RL benchmark:
- **Goal**: Balance a pole on a moving cart
- **State**: Cart position, cart velocity, pole angle, pole velocity (4 values)
- **Actions**: Push cart left or right (2 actions)
- **Reward**: +1 for each timestep the pole is balanced
- **Success**: Average reward ≥ 195 over 100 episodes

In [ ]:
def train_dqn_cartpole(n_episodes=500, max_steps=500, render_freq=None):
    """
    Train DQN agent on CartPole environment.
    """
    # Create environment
    env = gym.make('CartPole-v1')
    
    state_dim = env.observation_space.shape[0]  # 4
    action_dim = env.action_space.n  # 2
    
    print(f"CartPole Environment:")
    print(f"  State dimension: {state_dim}")
    print(f"  Action dimension: {action_dim}")
    print(f"  Goal: Keep pole balanced (reward ≥ 195)")
    print()
    
    # Create agent
    agent = DQNAgent(
        state_dim=state_dim,
        action_dim=action_dim,
        learning_rate=1e-3,
        discount_factor=0.99,
        epsilon_start=1.0,
        epsilon_end=0.01,
        epsilon_decay=0.995,
        buffer_size=10000,
        batch_size=64,
        target_update_freq=10
    )
    
    # Training loop
    episode_rewards = []
    losses = []
    
    for episode in range(n_episodes):
        state, _ = env.reset()
        total_reward = 0
        episode_loss = []
        
        for step in range(max_steps):
            # Select action
            action = agent.select_action(state)
            
            # Take action
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            
            # Store transition
            agent.store_transition(state, action, reward, next_state, done)
            
            # Learn
            loss = agent.learn()
            if loss is not None:
                episode_loss.append(loss)
            
            total_reward += reward
            state = next_state
            
            if done:
                break
        
        # Decay epsilon
        agent.decay_epsilon()
        
        # Record metrics
        episode_rewards.append(total_reward)
        if episode_loss:
            losses.append(np.mean(episode_loss))
        
        # Print progress
        if (episode + 1) % 50 == 0:
            avg_reward = np.mean(episode_rewards[-50:])
            print(f"Episode {episode + 1}/{n_episodes} | "
                  f"Avg Reward: {avg_reward:.1f} | "
                  f"Epsilon: {agent.epsilon:.3f}")
            
            # Check if solved
            if avg_reward >= 195:
                print(f"\n*** Environment SOLVED in {episode + 1} episodes! ***")
    
    env.close()
    return agent, episode_rewards, losses

# Train the agent
print("Training DQN on CartPole")
print("=" * 50)
agent_cartpole, rewards_cartpole, losses_cartpole = train_dqn_cartpole(n_episodes=300)

In [ ]:
# Visualize CartPole training
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Rewards
ax1 = axes[0]
ax1.plot(rewards_cartpole, alpha=0.3, label='Episode Reward')
window = 50
if len(rewards_cartpole) >= window:
    moving_avg = np.convolve(rewards_cartpole, np.ones(window)/window, mode='valid')
    ax1.plot(range(window-1, len(rewards_cartpole)), moving_avg, 
             label=f'{window}-Episode Moving Avg', linewidth=2)
ax1.axhline(y=195, color='g', linestyle='--', label='Solved Threshold (195)')
ax1.set_xlabel('Episode')
ax1.set_ylabel('Total Reward')
ax1.set_title('DQN CartPole Training - Rewards')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Loss
ax2 = axes[1]
if losses_cartpole:
    ax2.plot(losses_cartpole, alpha=0.5)
    if len(losses_cartpole) >= window:
        loss_avg = np.convolve(losses_cartpole, np.ones(window)/window, mode='valid')
        ax2.plot(range(window-1, len(losses_cartpole)), loss_avg, linewidth=2)
ax2.set_xlabel('Episode')
ax2.set_ylabel('Loss')
ax2.set_title('DQN CartPole Training - Loss')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print final statistics
print(f"\nFinal Statistics:")
print(f"  Last 100 episodes avg reward: {np.mean(rewards_cartpole[-100:]):.1f}")
print(f"  Best episode reward: {max(rewards_cartpole)}")
print(f"  Episodes to reach 195 avg: ", end="")
for i in range(100, len(rewards_cartpole)):
    if np.mean(rewards_cartpole[i-100:i]) >= 195:
        print(f"{i}")
        break
else:
    print("Not reached")

## 7. Policy Gradient (REINFORCE)

Instead of learning Q-values, **Policy Gradient** methods directly learn the policy (probability of actions).

### REINFORCE Algorithm

The policy gradient theorem:

$$\nabla J(\theta) = \mathbb{E}\left[ \sum_t \nabla \log \pi_\theta(a_t|s_t) \cdot G_t \right]$$

Where $G_t$ is the return (cumulative discounted reward) from time $t$.

### Key Differences from DQN

| Aspect | DQN | Policy Gradient |
|--------|-----|----------------|
| Output | Q-values | Action probabilities |
| Action selection | argmax Q | Sample from distribution |
| Update | After each step | After episode |
| Exploration | ε-greedy | Built-in (stochastic policy) |

In [ ]:
class PolicyNetwork(nn.Module):
    """
    Policy Network for REINFORCE.
    
    Outputs action probabilities (softmax).
    """
    
    def __init__(self, state_dim, action_dim, hidden_dim=128):
        super(PolicyNetwork, self).__init__()
        
        self.network = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, action_dim),
            nn.Softmax(dim=-1)  # Output probabilities
        )
    
    def forward(self, x):
        return self.network(x)


class REINFORCEAgent:
    """
    REINFORCE (Monte Carlo Policy Gradient) Agent.
    
    Learns policy directly by optimizing expected return.
    """
    
    def __init__(self, state_dim, action_dim, learning_rate=1e-3, discount_factor=0.99):
        self.gamma = discount_factor
        self.action_dim = action_dim
        
        # Policy network
        self.policy = PolicyNetwork(state_dim, action_dim).to(device)
        self.optimizer = optim.Adam(self.policy.parameters(), lr=learning_rate)
        
        # Episode memory
        self.log_probs = []
        self.rewards = []
    
    def select_action(self, state):
        """
        Select action by sampling from policy distribution.
        """
        state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)
        probs = self.policy(state_tensor)
        
        # Sample action from distribution
        dist = torch.distributions.Categorical(probs)
        action = dist.sample()
        
        # Store log probability for learning
        self.log_probs.append(dist.log_prob(action))
        
        return action.item()
    
    def store_reward(self, reward):
        """Store reward for current step."""
        self.rewards.append(reward)
    
    def learn(self):
        """
        Update policy using REINFORCE algorithm.
        
        Called at the end of each episode.
        """
        if not self.rewards:
            return 0
        
        # Calculate returns (discounted cumulative rewards)
        returns = []
        G = 0
        for reward in reversed(self.rewards):
            G = reward + self.gamma * G
            returns.insert(0, G)
        
        # Normalize returns for stability
        returns = torch.tensor(returns).to(device)
        returns = (returns - returns.mean()) / (returns.std() + 1e-8)
        
        # Calculate policy gradient loss
        policy_loss = []
        for log_prob, G in zip(self.log_probs, returns):
            # Negative because we want to maximize
            policy_loss.append(-log_prob * G)
        
        # Update policy
        self.optimizer.zero_grad()
        loss = torch.stack(policy_loss).sum()
        loss.backward()
        self.optimizer.step()
        
        # Clear episode memory
        self.log_probs = []
        self.rewards = []
        
        return loss.item()


def train_reinforce_cartpole(n_episodes=1000, max_steps=500):
    """
    Train REINFORCE agent on CartPole.
    """
    env = gym.make('CartPole-v1')
    
    state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.n
    
    print(f"Training REINFORCE on CartPole")
    print(f"=" * 50)
    
    agent = REINFORCEAgent(
        state_dim=state_dim,
        action_dim=action_dim,
        learning_rate=1e-3,
        discount_factor=0.99
    )
    
    episode_rewards = []
    
    for episode in range(n_episodes):
        state, _ = env.reset()
        total_reward = 0
        
        for step in range(max_steps):
            action = agent.select_action(state)
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            
            agent.store_reward(reward)
            total_reward += reward
            state = next_state
            
            if done:
                break
        
        # Learn at end of episode
        agent.learn()
        episode_rewards.append(total_reward)
        
        if (episode + 1) % 100 == 0:
            avg_reward = np.mean(episode_rewards[-100:])
            print(f"Episode {episode + 1}/{n_episodes} | Avg Reward: {avg_reward:.1f}")
            
            if avg_reward >= 195:
                print(f"\n*** Environment SOLVED! ***")
    
    env.close()
    return agent, episode_rewards

# Train REINFORCE agent
agent_reinforce, rewards_reinforce = train_reinforce_cartpole(n_episodes=500)

In [ ]:
# Compare DQN vs REINFORCE
fig, ax = plt.subplots(figsize=(12, 6))

window = 50

# DQN
if len(rewards_cartpole) >= window:
    dqn_avg = np.convolve(rewards_cartpole, np.ones(window)/window, mode='valid')
    ax.plot(range(window-1, len(rewards_cartpole)), dqn_avg, 
            label='DQN', linewidth=2, color='blue')

# REINFORCE
if len(rewards_reinforce) >= window:
    reinforce_avg = np.convolve(rewards_reinforce, np.ones(window)/window, mode='valid')
    ax.plot(range(window-1, len(rewards_reinforce)), reinforce_avg, 
            label='REINFORCE', linewidth=2, color='orange')

ax.axhline(y=195, color='g', linestyle='--', label='Solved (195)')
ax.set_xlabel('Episode')
ax.set_ylabel('Average Reward (50 episodes)')
ax.set_title('DQN vs REINFORCE on CartPole')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

print("\nComparison:")
print(f"  DQN final avg: {np.mean(rewards_cartpole[-100:]):.1f}")
print(f"  REINFORCE final avg: {np.mean(rewards_reinforce[-100:]):.1f}")

## 8. Pong-Style Game with DQN

Now let's create a simplified Pong-like game and train a DQN agent to play it!

### SimplePong Environment
- **State**: Ball position (x, y), ball velocity (vx, vy), paddle position
- **Actions**: Stay, Move Up, Move Down
- **Reward**: +1 for hitting ball, -1 for missing, small negative for each step

In [ ]:
class SimplePong:
    """
    Simplified Pong environment.
    
    Single player: keep the ball in play by moving paddle.
    """
    
    def __init__(self, width=20, height=10, paddle_size=3):
        self.width = width
        self.height = height
        self.paddle_size = paddle_size
        
        # State dimensions: ball_x, ball_y, ball_vx, ball_vy, paddle_y (normalized)
        self.state_dim = 5
        self.action_dim = 3  # Stay, Up, Down
        
        self.reset()
    
    def reset(self):
        """Reset game to initial state."""
        # Ball starts in center, moving toward paddle
        self.ball_x = self.width // 2
        self.ball_y = self.height // 2
        self.ball_vx = -1  # Moving toward paddle (left side)
        self.ball_vy = random.choice([-1, 0, 1])
        
        # Paddle on left side
        self.paddle_y = self.height // 2 - self.paddle_size // 2
        
        self.score = 0
        self.steps = 0
        
        return self._get_state()
    
    def _get_state(self):
        """Return normalized state."""
        return np.array([
            self.ball_x / self.width,
            self.ball_y / self.height,
            self.ball_vx / 2,  # Velocity normalized
            self.ball_vy / 2,
            self.paddle_y / self.height
        ], dtype=np.float32)
    
    def step(self, action):
        """
        Take action and update game state.
        
        Actions: 0=Stay, 1=Up, 2=Down
        """
        self.steps += 1
        reward = -0.01  # Small penalty per step
        done = False
        
        # Move paddle
        if action == 1:  # Up
            self.paddle_y = max(0, self.paddle_y - 1)
        elif action == 2:  # Down
            self.paddle_y = min(self.height - self.paddle_size, self.paddle_y + 1)
        
        # Move ball
        self.ball_x += self.ball_vx
        self.ball_y += self.ball_vy
        
        # Ball bounces off top/bottom walls
        if self.ball_y <= 0 or self.ball_y >= self.height - 1:
            self.ball_vy = -self.ball_vy
            self.ball_y = max(0, min(self.height - 1, self.ball_y))
        
        # Ball bounces off right wall
        if self.ball_x >= self.width - 1:
            self.ball_vx = -self.ball_vx
            self.ball_x = self.width - 1
        
        # Check if ball reaches left side (paddle side)
        if self.ball_x <= 1:
            # Check if paddle hits ball
            paddle_top = self.paddle_y
            paddle_bottom = self.paddle_y + self.paddle_size
            
            if paddle_top <= self.ball_y <= paddle_bottom:
                # Hit! Bounce back
                self.ball_vx = -self.ball_vx
                self.ball_x = 2
                # Add some randomness to vertical velocity
                self.ball_vy = random.choice([-1, 0, 1])
                reward = 1.0
                self.score += 1
            else:
                # Miss!
                reward = -1.0
                done = True
        
        # Max steps limit
        if self.steps >= 500:
            done = True
        
        return self._get_state(), reward, done, {'score': self.score}
    
    def render(self):
        """Text-based rendering of game."""
        grid = [['.' for _ in range(self.width)] for _ in range(self.height)]
        
        # Draw paddle
        for i in range(self.paddle_size):
            y = self.paddle_y + i
            if 0 <= y < self.height:
                grid[y][0] = '|'
        
        # Draw ball
        ball_y = int(max(0, min(self.height - 1, self.ball_y)))
        ball_x = int(max(0, min(self.width - 1, self.ball_x)))
        grid[ball_y][ball_x] = 'O'
        
        # Print
        print('╔' + '═' * self.width + '╗')
        for row in grid:
            print('║' + ''.join(row) + '║')
        print('╚' + '═' * self.width + '╝')
        print(f"Score: {self.score} | Steps: {self.steps}")

# Test Pong environment
print("SimplePong Environment")
print("=" * 30)
pong = SimplePong()
state = pong.reset()
print(f"State shape: {state.shape}")
print(f"State: {state}")
print()
pong.render()

In [ ]:
def train_dqn_pong(n_episodes=1000):
    """
    Train DQN agent on SimplePong.
    """
    env = SimplePong()
    
    print("Training DQN on SimplePong")
    print("=" * 50)
    
    agent = DQNAgent(
        state_dim=env.state_dim,
        action_dim=env.action_dim,
        learning_rate=1e-3,
        discount_factor=0.99,
        epsilon_start=1.0,
        epsilon_end=0.05,
        epsilon_decay=0.998,
        buffer_size=50000,
        batch_size=64,
        target_update_freq=20
    )
    
    episode_rewards = []
    episode_scores = []
    
    for episode in range(n_episodes):
        state = env.reset()
        total_reward = 0
        
        while True:
            action = agent.select_action(state)
            next_state, reward, done, info = env.step(action)
            
            agent.store_transition(state, action, reward, next_state, done)
            agent.learn()
            
            total_reward += reward
            state = next_state
            
            if done:
                break
        
        agent.decay_epsilon()
        episode_rewards.append(total_reward)
        episode_scores.append(info['score'])
        
        if (episode + 1) % 100 == 0:
            avg_reward = np.mean(episode_rewards[-100:])
            avg_score = np.mean(episode_scores[-100:])
            print(f"Episode {episode + 1}/{n_episodes} | "
                  f"Avg Reward: {avg_reward:.2f} | "
                  f"Avg Score: {avg_score:.1f} | "
                  f"Epsilon: {agent.epsilon:.3f}")
    
    return agent, episode_rewards, episode_scores

# Train Pong agent
agent_pong, rewards_pong, scores_pong = train_dqn_pong(n_episodes=500)

In [ ]:
# Visualize Pong training
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

window = 50

# Rewards
ax1 = axes[0]
ax1.plot(rewards_pong, alpha=0.3)
if len(rewards_pong) >= window:
    avg = np.convolve(rewards_pong, np.ones(window)/window, mode='valid')
    ax1.plot(range(window-1, len(rewards_pong)), avg, linewidth=2)
ax1.set_xlabel('Episode')
ax1.set_ylabel('Total Reward')
ax1.set_title('SimplePong Training - Rewards')
ax1.grid(True, alpha=0.3)

# Scores (ball hits)
ax2 = axes[1]
ax2.plot(scores_pong, alpha=0.3)
if len(scores_pong) >= window:
    avg = np.convolve(scores_pong, np.ones(window)/window, mode='valid')
    ax2.plot(range(window-1, len(scores_pong)), avg, linewidth=2, color='green')
ax2.set_xlabel('Episode')
ax2.set_ylabel('Score (Ball Hits)')
ax2.set_title('SimplePong Training - Score')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nFinal Statistics:")
print(f"  Avg Score (last 100): {np.mean(scores_pong[-100:]):.1f}")
print(f"  Best Score: {max(scores_pong)}")

In [ ]:
# Watch trained agent play Pong
print("Trained Agent Playing SimplePong")
print("=" * 40)

env = SimplePong()
state = env.reset()

# Disable exploration for testing
agent_pong.epsilon = 0

frames = []
while True:
    # Select best action
    with torch.no_grad():
        state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)
        q_values = agent_pong.policy_net(state_tensor)
        action = q_values.argmax().item()
    
    next_state, reward, done, info = env.step(action)
    
    # Store frame data
    frames.append({
        'ball': (env.ball_x, env.ball_y),
        'paddle': env.paddle_y,
        'score': info['score'],
        'action': ['Stay', 'Up', 'Down'][action]
    })
    
    state = next_state
    
    if done:
        break

# Show last few frames
print(f"\nGame ended with score: {info['score']}")
print(f"Total steps: {len(frames)}")

# Render final state
print("\nFinal game state:")
env.render()

## 9. Training Visualization

Let's create comprehensive visualizations of the learning process.

In [ ]:
# Comprehensive training comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

window = 50

# 1. GridWorld Q-Learning
ax = axes[0, 0]
ax.plot(rewards, alpha=0.3, color='blue')
if len(rewards) >= window:
    avg = np.convolve(rewards, np.ones(window)/window, mode='valid')
    ax.plot(range(window-1, len(rewards)), avg, linewidth=2, color='blue')
ax.set_title('1. Q-Learning on GridWorld')
ax.set_xlabel('Episode')
ax.set_ylabel('Reward')
ax.grid(True, alpha=0.3)

# 2. CartPole DQN
ax = axes[0, 1]
ax.plot(rewards_cartpole, alpha=0.3, color='green')
if len(rewards_cartpole) >= window:
    avg = np.convolve(rewards_cartpole, np.ones(window)/window, mode='valid')
    ax.plot(range(window-1, len(rewards_cartpole)), avg, linewidth=2, color='green')
ax.axhline(y=195, color='r', linestyle='--', alpha=0.7)
ax.set_title('2. DQN on CartPole')
ax.set_xlabel('Episode')
ax.set_ylabel('Reward')
ax.grid(True, alpha=0.3)

# 3. CartPole REINFORCE
ax = axes[1, 0]
ax.plot(rewards_reinforce, alpha=0.3, color='orange')
if len(rewards_reinforce) >= window:
    avg = np.convolve(rewards_reinforce, np.ones(window)/window, mode='valid')
    ax.plot(range(window-1, len(rewards_reinforce)), avg, linewidth=2, color='orange')
ax.axhline(y=195, color='r', linestyle='--', alpha=0.7)
ax.set_title('3. REINFORCE on CartPole')
ax.set_xlabel('Episode')
ax.set_ylabel('Reward')
ax.grid(True, alpha=0.3)

# 4. SimplePong DQN
ax = axes[1, 1]
ax.plot(scores_pong, alpha=0.3, color='purple')
if len(scores_pong) >= window:
    avg = np.convolve(scores_pong, np.ones(window)/window, mode='valid')
    ax.plot(range(window-1, len(scores_pong)), avg, linewidth=2, color='purple')
ax.set_title('4. DQN on SimplePong (Score)')
ax.set_xlabel('Episode')
ax.set_ylabel('Score (Ball Hits)')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 10. Summary

### Algorithms Implemented

| Algorithm | Type | Key Idea | Best For |
|-----------|------|----------|----------|
| **Q-Learning** | Value-based | Learn Q(s,a) table | Small, discrete states |
| **DQN** | Value-based | Neural network Q-function | Large/continuous states |
| **REINFORCE** | Policy-based | Direct policy optimization | Continuous actions |

### Key Concepts Learned

1. **RL Framework**: Agent, Environment, State, Action, Reward
2. **Exploration vs Exploitation**: ε-greedy strategy
3. **Value Functions**: Q(s,a) estimates expected return
4. **Experience Replay**: Breaks correlation, improves sample efficiency
5. **Target Networks**: Stabilizes training in deep RL
6. **Policy Gradients**: Directly optimize policy parameters

### Environments Conquered

1. **GridWorld**: Simple navigation (Q-Learning)
2. **CartPole**: Classic control (DQN, REINFORCE)
3. **SimplePong**: Game playing (DQN)

### Next Steps

- **Double DQN**: Reduce overestimation
- **Dueling DQN**: Separate value and advantage
- **PPO**: More stable policy gradients
- **A3C**: Parallel training
- **Atari Games**: Full pixel-based learning

In [ ]:
# Final summary of trained agents
print("="*60)
print("REINFORCEMENT LEARNING GAME - FINAL SUMMARY")
print("="*60)

print("""
Algorithms Implemented:
───────────────────────
1. Q-Learning (Tabular)
   - Bellman equation: Q(s,a) ← Q(s,a) + α[r + γ·max(Q(s')) - Q(s,a)]
   - Works for small, discrete state spaces
   
2. Deep Q-Network (DQN)
   - Neural network approximates Q-function
   - Experience replay for stable learning
   - Target network for stable Q-targets
   
3. REINFORCE (Policy Gradient)
   - Directly learns policy π(a|s)
   - Updates using: ∇J = E[∇log π(a|s) · G]
   - Monte Carlo returns for credit assignment

Environments Trained:
────────────────────
""")

print(f"1. GridWorld (Q-Learning)")
print(f"   Final avg reward: {np.mean(rewards[-100:]):.3f}")

print(f"\n2. CartPole (DQN)")
print(f"   Final avg reward: {np.mean(rewards_cartpole[-100:]):.1f}")
print(f"   Solved: {'Yes' if np.mean(rewards_cartpole[-100:]) >= 195 else 'Getting there'}")

print(f"\n3. CartPole (REINFORCE)")
print(f"   Final avg reward: {np.mean(rewards_reinforce[-100:]):.1f}")

print(f"\n4. SimplePong (DQN)")
print(f"   Final avg score: {np.mean(scores_pong[-100:]):.1f} ball hits")
print(f"   Best score: {max(scores_pong)} ball hits")

print("""
Key Takeaways:
─────────────
• RL learns through trial and error (no labeled data needed)
• Exploration is crucial early, exploitation later
• Experience replay breaks correlation in training
• Neural networks enable RL on complex state spaces
• Different algorithms suit different problems
""")